In [1]:
#pip install selenium
!pip install unidecode
!pip install selenium

In [2]:
#import libraries
import requests
from bs4 import BeautifulSoup
import lxml
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import numpy as np
from unidecode import unidecode
from urllib.parse import quote

In [3]:
#Read Andrew's IMDB data
df = pd.read_csv('Prophethuge (1).csv', index_col = 0)

#Create dataframe of only the stars from each movie, ensure that there are no duplicates
stars_list = df.drop(columns=df.columns.difference(['Star 1','Star 2','Star 3']))
stars_list = pd.concat([stars_list['Star 1'], stars_list['Star 2'], stars_list['Star 3']]).unique()
stars_tenure = pd.DataFrame({'Stars': stars_list, 'Years Active':'0', 'Tenure':0})
stars_tenure

,Stars,Years Active,Tenure
0,Brad Pitt,0,0
1,Tom Hanks,0,0
2,Jennifer Lopez,0,0
3,Naomi Ackie,0,0
4,Kristine Hartgen,0,0
...,...,...,...
10318,Danielle Darrieux,0,0
10319,Polly Walker,0,0
10320,Corey Burton,0,0
10321,Sung Kang,0,0


In [4]:
#Scrape wiki for the years active and add to a new column

#loop through every row from list and create url parts
for i, idx in enumerate(stars_tenure.index):
    name = str(stars_tenure.loc[idx]['Stars'])  # Convert to string if not already a string
    pattern = re.compile(r'[^A-Za-z0-9 ]+.-', re.UNICODE)
    name = re.sub(pattern, '', unidecode(name)).replace(' ', '_')
    typ = 'wiki'
    actor = quote('(actor)')
    actress = quote('(actress)')
    
    #replaces with ' (apostrophe)
    if '&apos;' in name:
        name = name.replace('&apos;', "'")
    
    # Construct URLs
    URL = f'https://en.wikipedia.org/{typ}/{name}'
    URL2 = f"https://en.wikipedia.org/{typ}/{name}_{actor}"
    URL3 = f"https://en.wikipedia.org/{typ}/{name}_{actress}"

    # Fetch content from URL
    print(f'Processing: {URL}')
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    section = soup.find('th', {'scope': 'row', 'class': 'infobox-label'}, string=['Years&nbsp;active', 'Years\xa0active', 'Years Active', 'Years active'])     
    
    print(f'Processing: {URL2}')
    page2 = requests.get(URL2)
    soup2 = BeautifulSoup(page2.content, 'html.parser')
    section2 = soup2.find('th', {'scope': 'row', 'class': 'infobox-label'}, string=['Years&nbsp;active', 'Years\xa0active', 'Years Active', 'Years active'])      
    
    print(f"Processing: {URL3}")
    page3 = requests.get(URL3)
    soup3 = BeautifulSoup(page3.content, 'html.parser')
    section3 = soup3.find('th', {'scope': 'row', 'class': 'infobox-label'}, string=['Years&nbsp;active', 'Years\xa0active', 'Years Active', 'Years active'])      
    
    #check which url variant returns data, then set it to current section
    if section != None:
        selected_section = section
        to_get = selected_section.find_next('td', {'class': 'infobox-data'})
        date_text = to_get.get_text(strip=True) if to_get else None
        stars_tenure.loc[idx,'Years Active'] = date_text
    
    elif section2 != None:
        selected_section = section2
        to_get = selected_section.find_next('td', {'class': 'infobox-data'})
        date_text = to_get.get_text(strip=True) if to_get else None
        stars_tenure.loc[idx,'Years Active'] = date_text
        
    elif section3 != None:
        selected_section = section3
        to_get = selected_section.find_next('td', {'class': 'infobox-data'})
        date_text = to_get.get_text(strip=True) if to_get else None
        stars_tenure.loc[idx,'Years Active'] = date_text
   
    # Error handling for titles that cannot be found, assign a value of 0
    
    else:
        stars_tenure.loc[idx,'Tenure'] = 0
        print(f'Could not scrape data for: {stars_tenure.loc[idx]["Stars"]}')
        print(f'Progress: {round((i+1)/len(stars_tenure) * 100, 2)}%')
        continue
    
    #Read and split year data as a string
        
    date_pattern = re.compile(r'(\d{4}–(?:present|\d{4}))\s*(?:\(([^)]*)\))?', re.IGNORECASE)
        
    #Find in the input string
    date_actor = date_pattern.findall(date_text)
    
    if date_actor:
        
        #check to see if more than one date range given on wiki
        if len(date_actor) > 1:
        
            extracted_date = None
            total_years = 0
            current_year = 2023
            
            #for each date range, check if it specifies the job, if it doesnt, treat it as an interger range
            for date, job in date_actor:

                if 'acting' in job.lower() or 'actor' in job.lower() or 'actress' in job.lower():
                    extracted_date = date
                    start_year, end_year = map(str, extracted_date.split('–'))

                    if end_year.lower() == "present":
                        end_year = current_year
                        start_year = int(start_year)
                        total_years = end_year - start_year

                    else:
                        print('Contents is dead 1')
                else:
                    start_year, end_year = map(str, date.split('–'))
                    start_year = int(start_year)
                    
                    #if the ending of the range is a number or not
                    if end_year.lower() != 'present':
                        end_year = int(end_year)
                        diff = end_year - start_year
                        total_years = total_years + diff

                    elif end_year.lower() =='present':
                        end_year = current_year
                        diff = end_year - start_year
                        total_years = total_years + diff
                            
                    else: 
                        print('Contents is dead 2')
                                    
            # Calculate the number of years
            print('tenure',total_years)

            stars_tenure.loc[idx,'Tenure'] = total_years
            print(stars_tenure.loc[idx, ['Stars','Years Active','Tenure']])
            print(f'Progress: {round((i+1)/len(stars_tenure) * 100, 2)}%')

        #if only one range is given, check to see if it ends in integer or not                  
        else:
            extracted_date = date_actor[0][0]

            current_year = 2023
            start_year, end_year = map(str, extracted_date.split('–'))
            start_year = int(start_year)        

            if end_year.lower() == "present":
                end_year = current_year
                # Calculate the number of years
                tenure = end_year - start_year
                # print('tenure',tenure)
                stars_tenure.loc[idx,'Tenure'] = tenure
                print(stars_tenure.loc[idx, ['Stars','Years Active','Tenure']])
                print(f'Progress: {round((i+1)/len(stars_tenure) * 100, 2)}%')

            else:
                end_year = int(end_year)
                # Calculate the number of years
                tenure = end_year - start_year
                # print('tenure',tenure)
                stars_tenure.loc[idx,'Tenure'] = tenure
                print(stars_tenure.loc[idx, ['Stars','Years Active','Tenure']])
                print(f'Progress: {round((i+1)/len(stars_tenure) * 100, 2)}%')
    
    #if none of these, it may have found the wiki page but there is no range for their career stated
    else:
        print('Script content is empty')

#save to new csv
stars_tenure.to_csv('actor_list.csv', index=False)  # Save the updated DataFrame to a CSV file

Processing: https://en.wikipedia.org/wiki/Brad_Pitt
Processing: https://en.wikipedia.org/wiki/Brad_Pitt_%28actor%29
Processing: https://en.wikipedia.org/wiki/Brad_Pitt_%28actress%29
Stars              Brad Pitt
Years Active    1987–present
Tenure                    36
Name: 0, dtype: object
Progress: 0.01%
Processing: https://en.wikipedia.org/wiki/Tom_Hanks
Processing: https://en.wikipedia.org/wiki/Tom_Hanks_%28actor%29
Processing: https://en.wikipedia.org/wiki/Tom_Hanks_%28actress%29
Stars              Tom Hanks
Years Active    1977–present
Tenure                    46
Name: 1, dtype: object
Progress: 0.02%
Processing: https://en.wikipedia.org/wiki/Jennifer_Lopez
Processing: https://en.wikipedia.org/wiki/Jennifer_Lopez_%28actor%29
Processing: https://en.wikipedia.org/wiki/Jennifer_Lopez_%28actress%29
Stars           Jennifer Lopez
Years Active      1986–present
Tenure                      37
Name: 2, dtype: object
Progress: 0.03%
Processing: https://en.wikipedia.org/wiki/Naomi_Ackie
P

In [5]:
#check to make sure all values populated
len(stars_tenure[stars_tenure['Tenure'].isnull()])

0

In [6]:
stars_tenure


,Stars,Years Active,Tenure
0,Brad Pitt,1987–present,36
1,Tom Hanks,1977–present,46
2,Jennifer Lopez,1986–present,37
3,Naomi Ackie,2009–present,14
4,Kristine Hartgen,0,0
...,...,...,...
10318,Danielle Darrieux,1931–2010,79
10319,Polly Walker,1989–present,34
10320,Corey Burton,1972–present,51
10321,Sung Kang,1999–present,24
